In [1]:
import pandas as pd
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/root/jupyter/SAs/batdongsan-datalake-v0-d1b3b2296e65.json'
from google.cloud import bigquery
bigquery_client = bigquery.Client(project='batdongsan-datalake-v0')
import numpy as np
pd.set_option('mode.chained_assignment', None)
import datetime as dt

In [ ]:
# We can read csv from the available data - which is extracted from the SQL query below
#data = pd.read_csv('abnormal trend_data raw.csv')

In [57]:
query = """
SELECT * FROM `batdongsan-datalake-v0.data_validation.event_summary_daily`
WHERE visit_date >= date_add(current_date(), interval -100 day)
--and visit_date <= '2022-07-18'
AND event_category NOT IN ('Outbound traffic','Core Web Vitals','Javascript error','Logging','Enhanced Ecommerce','Version','','Social Share')
AND event_category is not null
"""

In [58]:
job = bigquery_client.query(query)
df0 = job.to_dataframe()

In [59]:
df = df0[(df0['visit_date']<= dt.date(2022,7,26))]

In [60]:
df.head()

,visit_date,device_category,page_type,event_category,event_action,count_events,holiday,weekend,day_off
1980,2022-06-01,desktop,LDP,Engagement,Share LDP,300,0,0,0
1981,2022-06-01,mobile,LDP,Engagement,Share LDP,528,0,0,0
1982,2022-06-01,tablet,LDP,Engagement,Share LDP,10,0,0,0
1983,2022-06-01,desktop,Other Consumer Pages,Conversion,Lead-Phone,27,0,0,0
1984,2022-06-01,desktop,LDP,Conversion,Lead-Phone,28226,0,0,0


In [61]:
#Chỉ lấy dữ liệu của week_day hoặc ngày lễ để so sánh với nhau
dfw=df[(df['weekend']==0)&(df['event_category']!='undefined')]
df1 = dfw[dfw['event_action'].str.contains('Scroll|Display')==False]

In [56]:
dfa = df.groupby(['visit_date','event_action'])['count_events'].sum()
dfa

visit_date  event_action                  
2022-06-01  Activate-Auto Renew                 781
            Buy Pro Account Result               91
            Click Request Contact button       1048
            Click-Address Detail Input LCP     1057
            Click-Address Suggestion           3312
                                              ...  
2022-07-26  View Expired Listing LDP           4530
            Zoom-Location Insight PDP          1156
            click-boost-hour-selection         6295
            click-boost-listing-menu          36192
            result-field-upi                      6
Name: count_events, Length: 6162, dtype: int64

In [52]:
dfb = df1.unstack(-1).fillna(0)

In [77]:
dfb.columns

Index(['Activate-Auto Renew', 'Add-Custom Location Insight PDP',
       'Buy Pro Account Result', 'Click Request Contact button',
       'Click-Address Detail Input LCP', 'Click-Address Suggestion',
       'Click-Apply Filter Multisearch Consumer',
       'Click-Auto Extend Pro Account',
       'Click-Auto Extend Pro Account Disable Form Accept',
       'Click-Auto Extend Pro Account Disable Form Cancel',
       ...
       'Upgrade Pro Account Result', 'View Expired Listing LDP',
       'Zoom-Location Insight PDP', 'click-boost-hour-selection',
       'click-boost-listing-menu', 'listing-direct-pdp',
       'multisearch-filter-apply', 'result-field-upi', 'undefined',
       'view-sale-policy-pdp'],
      dtype='object', name='event_action', length=193)

In [54]:
present_data = dfb[dfb.index==dfb.index.max()]
present_data

event_action,Activate-Auto Renew,Add-Custom Location Insight PDP,Buy Pro Account Result,Click Request Contact button,Click-Address Detail Input LCP,Click-Address Suggestion,Click-Apply Filter Multisearch Consumer,Click-Auto Extend Pro Account,Click-Auto Extend Pro Account Disable Form Accept,Click-Auto Extend Pro Account Disable Form Cancel,...,Upgrade Pro Account Result,View Expired Listing LDP,Zoom-Location Insight PDP,click-boost-hour-selection,click-boost-listing-menu,listing-direct-pdp,multisearch-filter-apply,result-field-upi,undefined,view-sale-policy-pdp
visit_date,,,,,,,,,,,,,,,,,,,,,
2022-07-26,695.0,3.0,71.0,953.0,788.0,2747.0,5543.0,23.0,17.0,0.0,...,7.0,4530.0,1156.0,6295.0,36192.0,0.0,0.0,6.0,0.0,0.0


In [55]:
past_data = dfb[dfb.index<dfb.index.max()]

In [29]:
raw=past_data['Activate-Auto Renew']

In [28]:
type(past_data)

pandas.core.frame.DataFrame

In [41]:
type(raw)

pandas.core.series.Series

In [34]:
avg=raw.rolling(5,3).mean().fillna(0)
avg

visit_date
2022-06-01      0.000000
2022-06-02      0.000000
2022-06-03    867.666667
2022-06-04    781.750000
2022-06-05    684.400000
2022-06-06    693.600000
2022-06-07    660.000000
2022-06-08    664.600000
2022-06-09    744.200000
2022-06-10    852.800000
2022-06-11    809.600000
2022-06-12    699.000000
2022-06-13    730.400000
2022-06-14    723.600000
2022-06-15    736.600000
2022-06-16    801.600000
2022-06-17    877.800000
2022-06-18    808.800000
2022-06-19    695.800000
2022-06-20    710.000000
2022-06-21    696.800000
2022-06-22    683.800000
2022-06-23    634.600000
2022-07-10    570.000000
2022-07-11    375.200000
2022-07-12    201.200000
2022-07-13    151.600000
2022-07-14    265.200000
2022-07-15    435.400000
2022-07-16    578.600000
2022-07-17    637.000000
2022-07-18    703.000000
2022-07-19    688.600000
2022-07-20    697.800000
2022-07-21    728.200000
2022-07-22    823.400000
2022-07-23    775.000000
2022-07-24    666.200000
2022-07-25    640.000000
Name: Activate

In [39]:
type(avg[avg.index == avg.index.max()])

pandas.core.series.Series

In [42]:
present_data['Activate-Auto Renew'][0]

695.0

In [67]:
def abnormal_trend(df,group,measure=['count_events'],abs_dif_allowed=1000):
    df1 = df.groupby(group)[measure].sum()
    df2 = df1.unstack(-1).fillna(0)
    df2.columns = df2.columns.levels[1]
    
    present_data = df2[df2.index==df2.index.max()]
    past_data = df2[df2.index<df2.index.max()]
    measure = df2.columns

    lst = []
#     err = pd.DataFrame(columns = ['name','dif_perc','dif','benchmark_std'])

    for i in(measure):
        raw = past_data[i]
        avg = raw.rolling(5,min_periods = 3).mean()
        std = raw.rolling(10,min_periods = 5).std()/avg
        
        benchmark = avg[avg.index == avg.index.max()][0]
        benchmark_std = std[std.index == std.index.max()][0]*100
        
        data = present_data[i][0]

        dif = data - benchmark
        dif_perc = dif/(data+0.01)*100

        if np.abs(dif) > abs_dif_allowed:
            if np.abs(dif_perc) > 20:
                lst.append([i,dif_perc,dif,benchmark_std])
            elif np.abs(dif_perc) < 5:
                pass
            elif np.abs(dif_perc) > benchmark_std*2:
                lst.append([i,dif_perc,dif,benchmark_std])
            else:
                pass
        else:
            pass
    err_table = pd.DataFrame(lst)
    if err_table.shape[0] == 0:
        pass
    else:
        err_table.columns = ['name','dif_perc','dif','benchmark_std']
    err_table['date'] = df2.index.max()
    return err_table

In [68]:
er_event_category = abnormal_trend(df1,group=['visit_date','event_category'])
er_event_category.shape[0]

1

In [69]:
er_event_category

,name,dif_perc,dif,benchmark_std,date
0,Listing Management,-24.448363,-30348.0,12.637626,2022-07-26


In [70]:
er_event_action = abnormal_trend(df1,group=['visit_date','event_action'])
er_event_action.shape[0]

3

In [71]:
er_event_action

,name,dif_perc,dif,benchmark_std,date
0,Click-Confirm Up Listing LMP,-64.706217,-14681.2,16.732237,2022-07-26
1,Click-Quick Filter Tab LMP,-33.907556,-4373.4,16.845603,2022-07-26
2,Click-Up Listing LMP,-43.613619,-12385.4,11.449284,2022-07-26


In [1]:
#BQ export function
def post_to_bq(df,destination_table,write_type='append'):
    if df.shape[0] == 0:
        pass
    else:
        pandas_gbq.to_gbq(
            df, destination_table, project_id='batdongsan-datalake-v0', if_exists=write_type,
            table_schema = [{'name':'date','type': 'DATE'}]
        )

In [ ]:
#concat abnormal data from different devices (all/desktop/mobile)
eec = abnormal_trend(df1,group=['visit_date','event_category'])
eec['device'] = 'all'

eecd = abnormal_trend(df1[df1['device_category']=='desktop'],group=['visit_date','event_category'])
eecd['device'] = 'desktop'

eecm = abnormal_trend(df1[df1['device_category']!='desktop'],group=['visit_date','event_category'])
eecm['device'] = 'mobile'

eec_all = pd.concat([eec,eecd,eecm])

In [ ]:
#export to BQ
#post_to_bq(eec_all, destination_table = 'data_validation.event_category_error')